In [61]:
%reset -f
from Helper import *
%matplotlib widget
%jsroot on

a = Ana("quadMcp", "run530167", 1)
p = Par(a.runName)
c = Canva()

In [62]:
c.new(800,300)
hq1 = a.geth(F"{p.q1}", "250,0,500", p.fidLarge, "")
hq0 = a.geth(F"{p.q0}", "250,0,500", p.fidLarge, "same")
hq1.SetLineColor(kRed)

In [63]:
c.new(800,300)
pk0 = a.geth("peak[2]", "450,0,900", p.fidLarge, "")
pk1 = a.geth("peak[3]", "450,0,900", p.fidLarge, "same")
pk0.SetStats(0)
pk1.SetStats(0)
pk0.SetLineColor(kRed)

In [64]:
cCutSipm = "charge[0] > 50 && charge[1] > 50"
cutMcpPeak = "peak[2] > 100 && peak[2] < 850 && peak[3] > 100 && peak[3] < 850"
mcpSigmaDtCut = 0.1

qmin, qmax = 100, 180 
qminq, qmax1 = 150, 200 

qCut = p.cutRange2(p.q0,p.q1,qmin,qmax)
qCutCharge = p.cutRange2(p.q0,p.q1,qminq,qmax1)

In [65]:
c.new()
hfidl = a.geth(F"{p.yCry}:{p.xCry}", "100,3,8,100,3,8", p.fidMiddle, "zcol")


In [66]:
hfidl = a.geth(F"{p.yCry}:{p.xCry}", "100,3,8,100,3,8", cCutSipm, "zcol")
c.new(600,300).SetLogz(0)
FormatH(hfidl, a.runName, "track at cry X [cm]", "track at cry Y [cm]", 0)

box = TBox(p.fidlarge[0], p.fidlarge[2], p.fidlarge[1], p.fidlarge[3])
box.SetLineColor(kRed); box.SetLineWidth(3); box.SetFillStyle(0)

l1, l2 = TLine(p.fidmidx, p.fidlarge[2], p.fidmidx, p.fidlarge[3]), TLine(p.fidlarge[0], p.fidmidy, p.fidlarge[1], p.fidmidy)
l1.SetLineWidth(3); l2.SetLineWidth(3)
# l1.SetLineColor(kRed); l2.SetLineColor(kRed)
hfidl.Draw("zcol"); box.Draw("same"); l1.Draw("same"); l2.Draw("same")

In [67]:
c.new()
dtmcpall = a.geth(F"{p.dtMcp}", "1000,-1,1", F"{cutMcpPeak}")
fgaus = TF1("fgaus", "gaus")
dtmcpall.Fit(fgaus, "S")

cutMcpDt = F"abs({p.dtMcp}-{fgaus.GetParameter(1)}) < {mcpSigmaDtCut}"

c.new()
a.draw(F"{p.dtMcp}-{fgaus.GetParameter(1)}>>(1000,-0.5,0.5)", F"{cutMcpPeak} && {cutMcpDt}")

cutMcp = cutMcpPeak +p.cutAnd+ cutMcpDt

 FCN=1382.11 FROM MIGRAD    STATUS=CONVERGED      72 CALLS          73 TOTAL
                     EDM=7.50195e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.26247e+02   4.72371e+00   6.66035e-02   5.08085e-05
   2  Mean         5.03150e-01   2.45245e-04   3.95249e-06   1.31634e+00
   3  Sigma        4.30783e-02   1.76619e-04   1.68213e-05   1.43701e-01


In [68]:
# histoTimeRng = "200, 6.6, 7.4"
# histoQRng = "200,0.46,0.54"
# histoQmRng = "200,-0.06,0.0"

histoTimeRng = "200, 6.2, 7.8"
histoQRng = "200,0.4,0.6"
histoQmRng = "200,-0.1,0.1"

times1, times2, timesm = [],[],[]
q1, q2, qm = [],[],[]

for i in tqdm(range(4)):

    thisCut = qCut +p.cutAnd+ p.fidQuadrants[i] +p.cutAnd+ cutMcp
    times1.append( a.geth(F"{p.t0}-{p.mtMcp}", histoTimeRng, thisCut, "goff").Clone() )
    times2.append( a.geth(F"{p.t1}-{p.mtMcp}", histoTimeRng, thisCut, "goff").Clone() )
    timesm.append( a.geth(F"{p.mtSipm}-{p.mtMcp}", histoTimeRng, thisCut, "goff").Clone() )

    thisCut = qCutCharge +p.cutAnd+ p.fidQuadrants[i]
    q1.append( a.geth(F"0.5*{p.q0}/{p.mcSipm}", histoQRng, thisCut, "goff").Clone() )
    q2.append( a.geth(F"0.5*{p.q1}/{p.mcSipm}", histoQRng, thisCut, "goff").Clone() )
    qm.append( a.geth(F"{p.asymSipm}", histoQmRng, thisCut, "goff").Clone() )

100%|██████████| 4/4 [00:01<00:00,  3.46it/s]


In [72]:
cc = c.new(1000,800)
cc.Divide(2,2)

for idx, (h1, h2) in enumerate(zip(times1, times2)):
    cc.cd(idx+1)

    gStyle.SetFitFormat("g")
    gStyle.SetOptStat(1)
    gStyle.SetOptFit(11)
    h1.SetStats(1)
    h2.SetStats(1)
    h1.Fit("gaus", "Q")
    h2.Fit("gaus", "Q")
    
    # cc.SetRightMargin(0.13)
    # cc.SetLeftMargin(0.13)
    FormatH(h1, "", "t_{i}- MCP_{mt} [ns]", "Entries", 0)
    h1.SetLineColor(kBlue)
    # h1.GetYaxis().SetRangeUser(0, h1.GetMaximum()*1.05)
    h1.DrawClone("E")
    h2.SetLineColor(kBlack)
    h2.SetStats(0)
    h2.DrawClone("E sames")

c.savelast(a.saveas("tChanQuadrants"))

----------------------------------->>0
----------------------------------->>1
----------------------------------->>2
----------------------------------->>3
 FCN=64.5913 FROM MIGRAD    STATUS=CONVERGED      70 CALLS          71 TOTAL
                     EDM=2.73235e-13    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.49493e+02   5.67716e+00   1.71478e-02   8.85289e-09
   2  Mean         6.95941e+00   7.43021e-04   3.31850e-06   8.71621e-04
   3  Sigma        4.23519e-02   6.23040e-04   1.41736e-05   1.01905e-04
 FCN=53.6506 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=4.46495e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.33

Info in <TCanvas::Print>: pdf file out/run530167/quadMcp/tChanQuadrants.pdf has been created


In [58]:
cc = c.new(1000,800)
cc.Divide(2,2)

for idx, hh in enumerate(timesm):
    cc.cd(idx+1)

    gStyle.SetFitFormat("g")
    gStyle.SetOptStat(1)
    gStyle.SetOptFit(11)
    hh.SetStats(1)
    hh.Fit("gaus", "EMQ")

    cc.SetRightMargin(0.13)
    cc.SetLeftMargin(0.13)
    FormatH(hh, "", "t_{mean}- MCP_{mt} [ns]", "Entries", 0)

    hh.DrawClone("E")

c.savelast(a.saveas("tMeanQuadrants"))

Info in <TCanvas::Print>: pdf file out/run530164/quadMcp/tMeanQuadrants.pdf has been created


In [59]:
cc = c.new(1000,800)
cc.Divide(2,2)

for idx, (h1, h2) in enumerate(zip(q1, q2)):
    cc.cd(idx+1)

    gStyle.SetFitFormat("g")
    gStyle.SetOptStat(1)
    gStyle.SetOptFit(11)
    h1.SetStats(1)
    h2.SetStats(1)
    # h1.Fit("gaus", "Q")
    # h2.Fit("gaus", "Q")
    
    # cc.SetRightMargin(0.13)
    # cc.SetLeftMargin(0.13)
    FormatH(h1, "", "Q_{i} / (Q1+Q2)", "Entries", 0)
    h1.SetLineColor(kBlue)
    # h1.GetYaxis().SetRangeUser(0, h1.GetMaximum()*1.05)
    h1.DrawClone("E")
    h2.SetLineColor(kBlack)
    h2.SetStats(0)
    h2.DrawClone("E sames")

c.savelast(a.saveas("q1Quad"))

Info in <TCanvas::Print>: pdf file out/run530164/quadMcp/q1Quad.pdf has been created


In [60]:
cc = c.new(1000,800)
cc.Divide(2,2)

for idx, hh in enumerate(qm):
    cc.cd(idx+1)

    gStyle.SetFitFormat("g")
    gStyle.SetOptStat(1)
    gStyle.SetOptFit(11)
    hh.SetStats(1)
    # hh.Fit("gaus", "EMQ")

    cc.SetRightMargin(0.13)
    cc.SetLeftMargin(0.13)
    FormatH(hh, "", "(Q2-Q1)/(Q1+Q2)", "Entries", 0)

    hh.DrawClone("E")

c.savelast(a.saveas("mqQuad"))

Info in <TCanvas::Print>: pdf file out/run530164/quadMcp/mqQuad.pdf has been created
